#### 사용자 정의 함수

----
- 함수기능 : 모델 학습 진행 함수
- 함수이름 : training 
- 매개변수 : 함수 구동 시 필요한 재료(구동 할때마다 달라지는 값)  
    * 모델인스턴스  
    * 학습데이터셋 : 피쳐와 타겟(학습 전 Tensor화)  
    * 손실함수 인스턴스  
    * 최적화 인스턴스  
    * 학습 횟수 : 에포크  
    * 배치 크기 : BATCH_SIZE -> 1 에코프당 학습 횟수 
    * 배치 개수 : BATCH_CNT  
    * 검증용 데이터셋 : 피쳐와 타겟(검증 전 Tensor화)
- 함수 결과 : 학습 시 에포크당 손실값과 성능 지표값, 검증에 대한 손실값과 성능 지표값
---


---
- 함수기능 : 에포크 단위 모델 학습 진행 함수
- 함수이름 : EpochTraing
- 매개변수 : 함수 구동 시 필요한 재료(구동 할때마다 달라지는 값)   
    * 모델 인스턴스    
    * 학습용 데이터셋 : feature,label(학습 전 Tensor화)  
    * 손실함수 인스턴스  
    * 최적화 인스턴스  
    * 배치 크기 : BATCH_SIZE -> 1 에코프당 학습 횟수  
    * 배치 개수 : BATCH_CNT   
- 함수결과 : 손실값, 성능 지표 값
---

In [2]:
## 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimizer
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [1]:
## 에코드 단위 학습 진행 후 손실값, 성능 지표 값 반환 함수
def EpochTraing(model, feature, target, lossFunc, scoreFunc, optimizer, BATCH_CNT, BATCH_SIZE = 32) :
    
    loss_total, score_total = 0,0

    # 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행
    for batch in range(BATCH_CNT) : 
        # 배치 크기만큼 데이터셋 추출 후 텐서화 진행(메모리 과다 사용 방지를 위해)
        start = batch * BATCH_SIZE
        end = start + BATCH_SIZE

        X_train = torch.FloatTensor(feature[start:end].values).to(DEVICE)
        y_train = torch.FloatTensor(target[start:end].values).to(DEVICE)

        # 학습 진행(예측값 뽑기)
        pre_y = model(X_train)

        # 손실 계산 
        loss = lossFunc(pre_y,y_train)
        loss_total += loss

        # 점수 추출
        score = scoreFunc(pre_y, y_train)
        score_total += score

        # 최적화 : w, b를 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 loss와 성능 지표값 반환
    # 테스트 및 성능 검증 함수 사용
    return loss_total/BATCH_CNT, 0
    


In [3]:
## 검증 및 테스트 진행 후 손실값과 성능지표값 반환 함수
def testing(model, feature, target, lossFunc, scoreFunc) :
    
    # 최적화 기능 비활성화 후 데이터셋 추출 후 학습 진행
    with torch.no_grad(): 
        
        # 데이터셋 텐서화 진행(메모리 과다 사용 방지를 위해)
        featureTS = torch.FloatTensor(feature.values).to(DEVICE)
        targetTS = torch.FloatTensor(target.values).to(DEVICE)

        # 학습 진행(예측값 뽑기)
        pre_y = model(featureTS).to(DEVICE)

        # 손실 계산 
        loss = lossFunc(pre_y,targetTS)
    
        # 점수 추출
        score = scoreFunc(pre_y, targetTS)

    # 손실값과 성능지표 반환
    return loss, score 

In [4]:
# 학습 진행
EPOCHS = 10
Train_val_loss = {'TRAIN':[], 'VAL':[]}
Train_val_score = {'TRAIN':[], 'VAL':{}}
for epoch in range(EPOCHS) :
    # 학습 진행
    train_loss, train_score = EpochTraing()
    # 검증 진행
    val_loss, val_score = testing()
    
    # 에포크당 학습 및 검증 결과 저저장
    Train_val_loss['TRAIN'].append(train_loss)
    Train_val_score['TRAIN'].append(train_score)
    Train_val_loss['VAL'].append(val_loss)
    Train_val_score['VAL'].append(val_score)    

    # 에포크당 학습 및 검증 결과 출력
    print(f'[{epoch}.{EPOCHS}]\n- [TRAIN_lOSS] : {train_loss} SCORE : {train_score}')
    print(f' - [VALID]LOSS : {val_loss}  [VALID]SCORE : {val_score}')


TypeError: EpochTraing() missing 6 required positional arguments: 'model', 'feature', 'target', 'lossFunc', 'optimizer', and 'BATCH_CNT'